# Pré-processamento de dados

## Carregar bibliotecas

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from IPython.display import Markdown
from string import punctuation
from pathlib import Path
import pandas as pd
import nltk

!python -m spacy download pt
import spacy
nlp = spacy.load('pt_core_news_sm')

nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package rslp to /home/amanda/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /home/amanda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/amanda/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Leitura do arquivo de dados

In [2]:
data_path = Path("../data/raw/data.csv")
df = pd.read_csv(data_path, sep=";")
df.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1050785521201541121,@Laranjito76 A pessoa certa para isso seria o ...,Fri Oct 12 16:29:25 +0000 2018,1,:)
1,1050785431955140608,"@behin_d_curtain Para mim, é precisamente o co...",Fri Oct 12 16:29:04 +0000 2018,1,:)
2,1050785401248645120,Vou fazer um video hoje... estou pensando em f...,Fri Oct 12 16:28:56 +0000 2018,1,:)
3,1050785370982547461,"aaaaaaaa amei tanto essas polaroids, nem sei e...",Fri Oct 12 16:28:49 +0000 2018,1,:)
4,1050785368902131713,Valoriza o coração do menininho que vc tem. El...,Fri Oct 12 16:28:49 +0000 2018,1,:)


## Carregar dados utilizando o parâmetro chunksize

In [3]:
df = pd.read_csv(data_path, sep=";", chunksize=1000)

## Stemming e lemmatization com stopwords

In [4]:
df_stemmer = []
df_lemma = []

for i,data in enumerate(df):
    # Correção do fuso horário
    data['tweet_date'] = pd.to_datetime(data['tweet_date'])
    data['tweet_date'] = data.tweet_date.dt.tz_convert('Brazil/East')
    
    if i == 0:
        display(Markdown("### Conjunto de dados tweet_text"))
        display(data['tweet_text'].head())
    
    # Bag of words
    countvec = CountVectorizer()
    
    # Transformando as frases em tokens
    bag_of_words = countvec.fit_transform(data.tweet_text.tolist())
    
    # Criando dataframe com as contagens das palavras
    bow = pd.DataFrame(bag_of_words.toarray(), columns=countvec.get_feature_names_out())
    
    # Palavras mais frequentes
    frequente = bow.sum()
    frequente.sort_values(ascending=False)
    
    # Tokenizando frases 
    word_tokens = data.apply(lambda row: word_tokenize(row["tweet_text"]), axis=1)
    
    if i == 0:
        display(Markdown("### Frases tokenizadas"))
        display(word_tokens.head())
        
        
    #Stemming 
    # Criando objeto stemmer
    stemmer = nltk.stem.RSLPStemmer()
        
    # Realizando stemmer nas frases
    frase_stemmer = word_tokens.apply(lambda x: [stemmer.stem(y) for y in x])
    
    # Adicionando frases stemming ao dataframe
    stemmer = data.drop(columns=['tweet_text'], axis=1)
    stemmer['tweet_text'] = frase_stemmer
    df_stemmer.append(stemmer)
    
    if i == 0:
        display(Markdown("### Stemming com stopwords"))
        display(frase_stemmer.head())
   
    # Criando lista com todas as palavras
    doc = word_tokens.apply(lambda x: nlp(str([y for y in x])))
    
    # Realizando lematização
    frase_lemma = doc.apply(lambda row: "".join([w.lemma_ for w in row]))
    lemma = data.drop(columns=['tweet_text'], axis=1)
    lemma['tweet_text'] = frase_lemma
    df_lemma.append(lemma)
    
    if i == 0:
        display(Markdown("### Lemmatization com stopwords"))
        display(frase_lemma.head())
    
stemming = pd.concat(df_stemmer)
stemming.to_csv("../data/processed/stemming_com_stopwords.csv")

lemmatization = pd.concat(df_lemma)
lemmatization.to_csv("../data/processed/lemmatization_com_stopwords.csv")

### Conjunto de dados tweet_text

0    @Laranjito76 A pessoa certa para isso seria o ...
1    @behin_d_curtain Para mim, é precisamente o co...
2    Vou fazer um video hoje... estou pensando em f...
3    aaaaaaaa amei tanto essas polaroids, nem sei e...
4    Valoriza o coração do menininho que vc tem. El...
Name: tweet_text, dtype: object

### Frases tokenizadas

0    [@, Laranjito76, A, pessoa, certa, para, isso,...
1    [@, behin_d_curtain, Para, mim, ,, é, precisam...
2    [Vou, fazer, um, video, hoje, ..., estou, pens...
3    [aaaaaaaa, amei, tanto, essas, polaroids, ,, n...
4    [Valoriza, o, coração, do, menininho, que, vc,...
dtype: object

### Stemming com stopwords

0    [@, laranjito76, a, pesso, cert, par, iss, ser...
1    [@, behin_d_curtain, par, mim, ,, é, precis, o...
2    [vou, faz, um, vide, hoj, ..., est, pens, em, ...
3    [aaaaaaa, ame, tant, ess, polaroid, ,, nem, se...
4    [valoriz, o, coraçã, do, menin, que, vc, tem, ...
dtype: object

### Lemmatization com stopwords

0    ['@','Laranjito76','o','pessoa','certo','para'...
1    ['@','behin_d_curtain','para','eu',',','ser','...
2    ['Vou','fazer','um','video','hoje','...','esta...
3    ['aaaaaaaar','ameir','tanto','esse','polaroids...
4    ['Valoriza','o','coração','de o','menininho','...
dtype: object

## Stemming e lemmatization sem stopwords

In [4]:
df_stemmer_sem_stopwords = []
df_lemma_sem_stopwords = []

stop_words = set(stopwords.words('portuguese') + list(punctuation))

for x,data in enumerate(df):
    # Correção do fuso horário
    data['tweet_date'] = pd.to_datetime(data['tweet_date'])
    data['tweet_date'] = data.tweet_date.dt.tz_convert('Brazil/East')
    
    if x == 0:
        display(Markdown("### Conjunto de dados tweet_text"))
        display(data['tweet_text'].head())
    
    # Tokenizando frases 
    word_tokens = data.apply(lambda row: word_tokenize(row["tweet_text"]), axis=1)
    if x == 0:
        display(Markdown("### Frases tokenizadas"))
        display(word_tokens.head())
    
    # Removendo stopwords
    frase_sem_stopword = word_tokens.apply(lambda x: [word for word in x if not word.lower() in stop_words])
    
    if x == 0:
        display(Markdown("### Frases tokenizadas sem stopwords"))
        display(frase_sem_stopword.head())
        
        
    #Stemming 
    # Criando objeto stemmer
    stemmer = nltk.stem.RSLPStemmer()
    
    # Realizando stemmer nas frases
    frase_stemmer_sem_stopword = frase_sem_stopword.apply(lambda x: [stemmer.stem(y) for y in x])

    # Adicionando frases stemming ao dataframe
    stemmer_sem_stopword = data.drop(columns=['tweet_text'], axis=1)
    stemmer_sem_stopword['tweet_text'] = frase_stemmer_sem_stopword
    df_stemmer_sem_stopwords.append(stemmer_sem_stopword)
    
    if x == 0:
        display(Markdown("### Stemming sem stopwords"))
        display(frase_stemmer_sem_stopword.head())
    
    # Criando lista com todas as palavras
    doc_sem_stopword = frase_sem_stopword.apply(lambda x: nlp(str([y for y in x])))
    
    # Realizando lematização
    frase_lema_sem_stopwords = doc_sem_stopword.apply(lambda row: "".join([w.lemma_ for w in row]))
    lemma_sem_stopwrds = data.drop(columns=['tweet_text'], axis=1)
    lemma_sem_stopwrds['tweet_text'] = frase_lema_sem_stopwords
    df_lemma_sem_stopwords.append(lemma_sem_stopwrds)
    
    if x == 0:
        display(Markdown("### Lemmatization sem stopwords"))
        display(frase_lema_sem_stopwords.head())
    
stemming_sem_stopwords = pd.concat(df_stemmer_sem_stopwords)
stemming_sem_stopwords.to_csv("../data/processed/stemming_sem_stopwords.csv")

lemmatization_sem_stopwords = pd.concat(df_lemma_sem_stopwords)
lemmatization_sem_stopwords.to_csv("../data/processed/lemmatization_sem_stopwords.csv")

### Conjunto de dados tweet_text

0    @Laranjito76 A pessoa certa para isso seria o ...
1    @behin_d_curtain Para mim, é precisamente o co...
2    Vou fazer um video hoje... estou pensando em f...
3    aaaaaaaa amei tanto essas polaroids, nem sei e...
4    Valoriza o coração do menininho que vc tem. El...
Name: tweet_text, dtype: object

### Frases tokenizadas

0    [@, Laranjito76, A, pessoa, certa, para, isso,...
1    [@, behin_d_curtain, Para, mim, ,, é, precisam...
2    [Vou, fazer, um, video, hoje, ..., estou, pens...
3    [aaaaaaaa, amei, tanto, essas, polaroids, ,, n...
4    [Valoriza, o, coração, do, menininho, que, vc,...
dtype: object

### Frases tokenizadas sem stopwords

0          [Laranjito76, pessoa, certa, vale, azevedo]
1    [behin_d_curtain, mim, precisamente, contrário...
2    [Vou, fazer, video, hoje, ..., pensando, falar...
3    [aaaaaaaa, amei, tanto, polaroids, sei, expres...
4    [Valoriza, coração, menininho, vc, diferente, ...
dtype: object

### Stemming sem stopwords

0              [laranjito76, pesso, cert, val, azeved]
1    [behin_d_curtain, mim, precis, contr, vem, chu...
2    [vou, faz, vide, hoj, ..., pens, fal, pouc, so...
3    [aaaaaaa, ame, tant, polaroid, sei, express, q...
4    [valoriz, coraçã, menin, vc, difer, faç, sorr,...
dtype: object

### Lemmatization sem stopwords

0    ['Laranjito76','pessoa','certo','valer','azeve...
1    ['behin_d_curtain','eu','precisamente','contrá...
2    ['Vou','fazer','video','hoje','...','pensar','...
3    ['aaaaaaaar','ameir','tanto','polaroids','sabe...
4    ['Valoriza','coração','menininho','vc','difere...
dtype: object